In [1]:
import numpy as np
import xarray as xr

In [3]:
def boxmean(da): 
    """
    Compute spatial weighted mean
    da      :  xarray DataArray
    """ 
    #weights = np.cos(np.deg2rad(da.lat)) 
    #weights.name = 'weights' 
    #boxmean = da.weighted(weights).mean(dim=('lat','lon')) 
    
    if hasattr(da, 'lat'):
        weights = np.cos(da.lat * np.pi / 180)
        boxmean = da.weighted(weights).mean(dim=('lat','lon')) 
    elif hasattr(da, 'latitude'):
        weights = np.cos(da.latitude * np.pi / 180)
        boxmean = da.weighted(weights).mean(dim=('latitude','longitude')) 
    
    return boxmean 

def NHboxmean(da): 
    """
    Compute spatial weighted mean for northern hemispere
    da      :  xarray DataArray
    """
    if hasattr(da, 'lat'): 
        weights = np.cos(da.lat * np.pi / 180)
        boxmean = da.sel(lat=slice(0,90)).weighted(weights).mean(dim=('lat','lon')) 
    elif hasattr(da, 'latitude'):
        weights = np.cos(da.latitude * np.pi / 180)
        boxmean = da.sel(latitude=slice(0,90)).weighted(weights).mean(dim=('latitude','longitude')) 
    
    return boxmean 

def EUboxmean(da): 
    """
    Compute spatial weighted mean for europe
    da      :  xarray DataArray
    """ 
    #weights = np.cos(np.deg2rad(da.lat)) 
    #weights.name = 'weights' 
    #boxmean = da.weighted(weights).mean(dim=('lat','lon')) 
    
    if hasattr(da, 'lat'):
        weights = np.cos(da.lat * np.pi / 180)
        boxmean = da.sel(lat=slice(30,70),lon=slice(-10,40)).weighted(weights).mean(dim=('lat','lon')) 
    elif hasattr(da, 'latitude'):
        weights = np.cos(da.latitude * np.pi / 180)
        boxmean = da.sel(latitude=slice(30,70),longitude=slice(-10,40)).weighted(weights).mean(dim=('latitude','longitude')) 
    
    return boxmean 


### Open the data from LENTIS PD and from 2K 

In [4]:
ds_tas_PD=xr.open_mfdataset('/usr/people/muntjewe/nobackup/nobackup_1/LENTIS/PD/Amon/tas/tas*.nc',combine='nested',concat_dim='ens')
tas_PD=ds_tas_PD['tas']

In [5]:
ds_tas_2K=xr.open_mfdataset('/usr/people/muntjewe/nobackup/nobackup_1/LENTIS/2K/Amon/tas/tas*.nc',combine='nested',concat_dim='ens')
tas_2K=ds_tas_2K['tas']

# align the time dimension, necessary for subtracting 2 xarray datasets
to check if it worked, run the following (this should not give an error): 
`xr.align(tas_PD, tas_2K, join='exact')`

In [6]:
tas_2K['time'] = tas_PD['time']

### compute and print seasonal mean and std dev of the differences

In [13]:

diff_season = tas_2K - tas_PD 
diff_mean = diff_season.mean(dim='ens').groupby('time.season').mean(dim='time')
diff_std = diff_season.std(dim='ens').groupby('time.season').mean(dim='time')

In [14]:
print('Now we are computing some statistics of the tas differences between PD an 2K (takes a long time!!')



print('Global')
## Explicit conversion by wrapping a DataArray with np.asarray also works. I need to do this to access values. See https://docs.xarray.dev/en/v0.9.3/dask.html
glbias = (np.asarray(boxmean(diff_mean))) 
glstd = (np.asarray(boxmean(diff_std))) 
print(str(boxmean(diff_mean).season[0].values)+' = '+str(np.round(glbias[0],2))+' (ens stddev: '+str(np.round(glstd[0],2))+')')
print(str(boxmean(diff_mean).season[1].values)+' = '+str(np.round(glbias[1],2))+' (ens stddev: '+str(np.round(glstd[1],2))+')')
print(str(boxmean(diff_mean).season[2].values)+' = '+str(np.round(glbias[2],2))+' (ens stddev: '+str(np.round(glstd[2],2))+')')
print(str(boxmean(diff_mean).season[3].values)+' = '+str(np.round(glbias[3],2))+' (ens stddev: '+str(np.round(glstd[3],2))+')')
print('ANN = '+str(np.round(glbias.mean(),2))+' (ens stddev: '+str(np.round(glstd.mean(),2))+')')
print('')
nhbias = (np.asarray(NHboxmean(diff_mean))) 
nhstd = (np.asarray(NHboxmean(diff_std))) 
print('Northern Hemisphere')
print(str(boxmean(diff_mean).season[0].values)+' = '+str(np.round(nhbias[0],2))+' (ens stddev: '+str(np.round(nhstd[0],2))+')')
print(str(boxmean(diff_mean).season[1].values)+' = '+str(np.round(nhbias[1],2))+' (ens stddev: '+str(np.round(nhstd[1],2))+')')
print(str(boxmean(diff_mean).season[2].values)+' = '+str(np.round(nhbias[2],2))+' (ens stddev: '+str(np.round(nhstd[2],2))+')')
print(str(boxmean(diff_mean).season[3].values)+' = '+str(np.round(nhbias[3],2))+' (ens stddev: '+str(np.round(nhstd[3],2))+')')
print('== ANN: '+str(np.round(nhbias.mean(),2))+' (ens stddev: '+str(np.round(nhstd.mean(),2))+')')
print('')
eubias = (np.asarray(EUboxmean(diff_mean))) 
eustd = (np.asarray(EUboxmean(diff_std))) 
print('Europe')
print(str(boxmean(diff_mean).season[0].values)+' = '+str(np.round(eubias[0],2))+' (ens stddev: '+str(np.round(eustd[0],2))+')')
print(str(boxmean(diff_mean).season[1].values)+' = '+str(np.round(eubias[1],2))+' (ens stddev: '+str(np.round(eustd[1],2))+')')
print(str(boxmean(diff_mean).season[2].values)+' = '+str(np.round(eubias[2],2))+' (ens stddev: '+str(np.round(eustd[2],2))+')')
print(str(boxmean(diff_mean).season[3].values)+' = '+str(np.round(eubias[3],2))+' (ens stddev: '+str(np.round(eustd[3],2))+')')
print('== ANN: '+str(np.round(eubias.mean(),2))+' (ens stddev: '+str(np.round(eustd.mean(),2))+')')

Now we are computing some statistics of the tas differences between PD an 2K (takes a long time!!
Global
DJF = 2.02 (ens stddev: 1.51)
JJA = 1.9 (ens stddev: 1.23)
MAM = 1.83 (ens stddev: 1.39)
SON = 2.06 (ens stddev: 1.26)
ANN = 1.95 (ens stddev: 1.35)

Northern Hemisphere
DJF = 2.67 (ens stddev: 1.99)
JJA = 2.42 (ens stddev: 1.37)
MAM = 2.29 (ens stddev: 1.72)
SON = 2.72 (ens stddev: 1.49)
== ANN: 2.52 (ens stddev: 1.64)

Europe
DJF = 2.4 (ens stddev: 2.58)
JJA = 2.81 (ens stddev: 1.86)
MAM = 2.29 (ens stddev: 2.16)
SON = 2.69 (ens stddev: 1.9)
== ANN: 2.55 (ens stddev: 2.13)


### for checking purposes

Also compute and print the absolute values of seasonal mean and std dev of the PD and the 2K 

In [9]:
tas_PD_season = tas_PD 
tas_PD_mean = tas_PD_season.mean(dim='ens').groupby('time.season').mean(dim='time')
tas_PD_std = tas_PD_season.std(dim='ens').groupby('time.season').mean(dim='time')



In [15]:
print('Now we are computing some statistics of the tas PD (takes a long time!!')



print('Global')
## Explicit conversion by wrapping a DataArray with np.asarray also works. I need to do this to access values. See https://docs.xarray.dev/en/v0.9.3/dask.html
glbias = (np.asarray(boxmean(tas_PD_mean))) 
glstd = (np.asarray(boxmean(tas_PD_std))) 
print(str(boxmean(tas_PD_mean).season[0].values)+' = '+str(np.round(glbias[0],2))+' (ens stddev: '+str(np.round(glstd[0],2))+')')
print(str(boxmean(tas_PD_mean).season[1].values)+' = '+str(np.round(glbias[1],2))+' (ens stddev: '+str(np.round(glstd[1],2))+')')
print(str(boxmean(tas_PD_mean).season[2].values)+' = '+str(np.round(glbias[2],2))+' (ens stddev: '+str(np.round(glstd[2],2))+')')
print(str(boxmean(tas_PD_mean).season[3].values)+' = '+str(np.round(glbias[3],2))+' (ens stddev: '+str(np.round(glstd[3],2))+')')
print('ANN = '+str(np.round(glbias.mean(),2))+' (ens stddev: '+str(np.round(glstd.mean(),2))+')')
print('')
nhbias = (np.asarray(NHboxmean(tas_PD_mean))) 
nhstd = (np.asarray(NHboxmean(tas_PD_std))) 
print('Northern Hemisphere')
print(str(boxmean(tas_PD_mean).season[0].values)+' = '+str(np.round(nhbias[0],2))+' (ens stddev: '+str(np.round(nhstd[0],2))+')')
print(str(boxmean(tas_PD_mean).season[1].values)+' = '+str(np.round(nhbias[1],2))+' (ens stddev: '+str(np.round(nhstd[1],2))+')')
print(str(boxmean(tas_PD_mean).season[2].values)+' = '+str(np.round(nhbias[2],2))+' (ens stddev: '+str(np.round(nhstd[2],2))+')')
print(str(boxmean(tas_PD_mean).season[3].values)+' = '+str(np.round(nhbias[3],2))+' (ens stddev: '+str(np.round(nhstd[3],2))+')')
print('== ANN: '+str(np.round(nhbias.mean(),2))+' (ens stddev: '+str(np.round(nhstd.mean(),2))+')')
print('')
eubias = (np.asarray(EUboxmean(tas_PD_mean))) 
eustd = (np.asarray(EUboxmean(tas_PD_std))) 
print('Europe')
print(str(boxmean(tas_PD_mean).season[0].values)+' = '+str(np.round(eubias[0],2))+' (ens stddev: '+str(np.round(eustd[0],2))+')')
print(str(boxmean(tas_PD_mean).season[1].values)+' = '+str(np.round(eubias[1],2))+' (ens stddev: '+str(np.round(eustd[1],2))+')')
print(str(boxmean(tas_PD_mean).season[2].values)+' = '+str(np.round(eubias[2],2))+' (ens stddev: '+str(np.round(eustd[2],2))+')')
print(str(boxmean(tas_PD_mean).season[3].values)+' = '+str(np.round(eubias[3],2))+' (ens stddev: '+str(np.round(eustd[3],2))+')')
print('== ANN: '+str(np.round(eubias.mean(),2))+' (ens stddev: '+str(np.round(eustd.mean(),2))+')')

Now we are computing some statistics of the tas PD (takes a long time!!
Global
DJF = 287.08 (ens stddev: 1.1)
JJA = 290.78 (ens stddev: 0.88)
MAM = 288.8 (ens stddev: 1.0)
SON = 289.0 (ens stddev: 0.93)
ANN = 288.91 (ens stddev: 0.97)

Northern Hemisphere
DJF = 282.73 (ens stddev: 1.46)
JJA = 294.57 (ens stddev: 0.97)
MAM = 287.44 (ens stddev: 1.24)
SON = 290.02 (ens stddev: 1.12)
== ANN: 288.69 (ens stddev: 1.2)

Europe
DJF = 277.81 (ens stddev: 1.99)
JJA = 293.97 (ens stddev: 1.38)
MAM = 283.82 (ens stddev: 1.62)
SON = 287.04 (ens stddev: 1.44)
== ANN: 285.66 (ens stddev: 1.6)


In [16]:

tas_2K_season = tas_2K
tas_2K_mean = tas_2K_season.mean(dim='ens').groupby('time.season').mean(dim='time')
tas_2K_std = tas_2K_season.std(dim='ens').groupby('time.season').mean(dim='time')

In [17]:
print('Now we are computing some statistics of the tas 2K (takes a long time!!')



print('Global')
## Explicit conversion by wrapping a DataArray with np.asarray also works. I need to do this to access values. See https://docs.xarray.dev/en/v0.9.3/dask.html
glbias = (np.asarray(boxmean(tas_2K_mean))) 
glstd = (np.asarray(boxmean(tas_2K_std))) 
print(str(boxmean(tas_2K_mean).season[0].values)+' = '+str(np.round(glbias[0],2))+' (ens stddev: '+str(np.round(glstd[0],2))+')')
print(str(boxmean(tas_2K_mean).season[1].values)+' = '+str(np.round(glbias[1],2))+' (ens stddev: '+str(np.round(glstd[1],2))+')')
print(str(boxmean(tas_2K_mean).season[2].values)+' = '+str(np.round(glbias[2],2))+' (ens stddev: '+str(np.round(glstd[2],2))+')')
print(str(boxmean(tas_2K_mean).season[3].values)+' = '+str(np.round(glbias[3],2))+' (ens stddev: '+str(np.round(glstd[3],2))+')')
print('ANN = '+str(np.round(glbias.mean(),2))+' (ens stddev: '+str(np.round(glstd.mean(),2))+')')
print('')
nhbias = (np.asarray(NHboxmean(tas_2K_mean))) 
nhstd = (np.asarray(NHboxmean(tas_2K_std))) 
print('Northern Hemisphere')
print(str(boxmean(tas_2K_mean).season[0].values)+' = '+str(np.round(nhbias[0],2))+' (ens stddev: '+str(np.round(nhstd[0],2))+')')
print(str(boxmean(tas_2K_mean).season[1].values)+' = '+str(np.round(nhbias[1],2))+' (ens stddev: '+str(np.round(nhstd[1],2))+')')
print(str(boxmean(tas_2K_mean).season[2].values)+' = '+str(np.round(nhbias[2],2))+' (ens stddev: '+str(np.round(nhstd[2],2))+')')
print(str(boxmean(tas_2K_mean).season[3].values)+' = '+str(np.round(nhbias[3],2))+' (ens stddev: '+str(np.round(nhstd[3],2))+')')
print('== ANN: '+str(np.round(nhbias.mean(),2))+' (ens stddev: '+str(np.round(nhstd.mean(),2))+')')
print('')
eubias = (np.asarray(EUboxmean(tas_2K_mean))) 
eustd = (np.asarray(EUboxmean(tas_2K_std))) 
print('Europe')
print(str(boxmean(tas_2K_mean).season[0].values)+' = '+str(np.round(eubias[0],2))+' (ens stddev: '+str(np.round(eustd[0],2))+')')
print(str(boxmean(tas_2K_mean).season[1].values)+' = '+str(np.round(eubias[1],2))+' (ens stddev: '+str(np.round(eustd[1],2))+')')
print(str(boxmean(tas_2K_mean).season[2].values)+' = '+str(np.round(eubias[2],2))+' (ens stddev: '+str(np.round(eustd[2],2))+')')
print(str(boxmean(tas_2K_mean).season[3].values)+' = '+str(np.round(eubias[3],2))+' (ens stddev: '+str(np.round(eustd[3],2))+')')
print('== ANN: '+str(np.round(eubias.mean(),2))+' (ens stddev: '+str(np.round(eustd.mean(),2))+')')

Now we are computing some statistics of the tas 2K (takes a long time!!
Global
DJF = 289.09 (ens stddev: 1.08)
JJA = 292.68 (ens stddev: 0.9)
MAM = 290.63 (ens stddev: 1.01)
SON = 291.06 (ens stddev: 0.89)
ANN = 290.86 (ens stddev: 0.97)

Northern Hemisphere
DJF = 285.4 (ens stddev: 1.4)
JJA = 296.99 (ens stddev: 1.01)
MAM = 289.73 (ens stddev: 1.24)
SON = 292.75 (ens stddev: 1.03)
== ANN: 291.22 (ens stddev: 1.17)

Europe
DJF = 280.24 (ens stddev: 1.74)
JJA = 296.76 (ens stddev: 1.35)
MAM = 286.1 (ens stddev: 1.49)
SON = 289.73 (ens stddev: 1.33)
== ANN: 288.21 (ens stddev: 1.48)
